In [15]:
import numpy as np
import pandas as pd

In [16]:
classesDict = {
    "T1":0,
    "T1_c":1,
    "T2":2,
    "T2-FLAIR":3,
    "FS":4,
    "PD":5,
    "SWI":6,
    "GRE":7,
    "T2*":8,
    "DWI":9,
    "ADC":10,
    "BOLD":11,
    "FlowSensitve":12,
    "PWI":13,
    "ASL":14,
    "Hippo":15,
    "DTI":16,
    "Other":17
}

In [17]:
labels_path = '/home/soenke/Internship/DeepDicomSortFork/prototyping/combinedDfs/CombinedTestingFrame_A2_A3_O3_egd_rss_hb_pruned.txt'

In [18]:
Labels = pd.read_csv(labels_path,names=['ID','label','extra'],sep='\t', dtype={'ID':str,'label':int,'extra':int})

In [19]:
Label_name_list = list(classesDict.keys())

In [20]:
Labels

,ID,label,extra
0,/trinity/home/r098375/DDS/data/ADNI/ADNI3/test...,3,0
1,/trinity/home/r098375/DDS/data/ADNI/ADNI3/test...,17,0
2,/trinity/home/r098375/DDS/data/ADNI/ADNI3/test...,3,0
3,/trinity/home/r098375/DDS/data/ADNI/ADNI3/test...,15,0
4,/trinity/home/r098375/DDS/data/ADNI/ADNI3/test...,3,0
...,...,...,...
66775,/trinity/home/r098375/DDS/data/HeartBrain/NIFT...,0,0
66776,/trinity/home/r098375/DDS/data/HeartBrain/NIFT...,0,0
66777,/trinity/home/r098375/DDS/data/HeartBrain/NIFT...,0,0
66778,/trinity/home/r098375/DDS/data/HeartBrain/NIFT...,0,0


In [21]:
def extractDataset(ID):
    if 'ADNI2' in ID:
        return 'ADNI2'
    elif 'ADNI3' in ID:
        return 'ADNI3'
    elif 'OASIS3' in ID:
        return 'OASIS3'
    elif 'egd' in ID:
        return 'egd'
    elif 'Rstudy' in ID:
        return 'rss'
    elif 'HeartBrain' in ID:
        return 'HeartBrain'
    return None

def assignStringLabel(label, labelList):
    return labelList[label]

In [22]:
FullLabelFrame = Labels.copy()

In [23]:
FullLabelFrame['dataset'] = FullLabelFrame['ID'].apply(extractDataset)
FullLabelFrame['string_label'] = FullLabelFrame['label'].apply(assignStringLabel, labelList=Label_name_list)

In [25]:
FullLabelFrame.groupby('dataset')['string_label'].value_counts()

dataset     string_label
ADNI2       PD               420
            T2               420
ADNI3       Other           8370
            T2*             5595
            T1              3570
            T2-FLAIR        3240
            Hippo           2955
            PWI             1455
            ASL              735
            DTI              120
            SWI               45
HeartBrain  Other           1305
            PWI              495
            ASL              465
            SWI              375
            T2-FLAIR         360
            Hippo            225
            T1               195
OASIS3      Other           8445
            BOLD            4965
            T2              3810
            T1              3450
            DWI             3240
            T2*             2175
            ASL             1620
            GRE             1440
            T2-FLAIR        1215
            SWI             1215
            FlowSensitve    1050
egd         PWI   

In [28]:
print(FullLabelFrame.groupby('string_label')['dataset'].value_counts().to_latex())

\begin{tabular}{llr}
\toprule
 &  & count \\
string_label & dataset &  \\
\midrule
ADC & egd & 300 \\
\cline{1-3}
\multirow[t]{3}{*}{ASL} & OASIS3 & 1620 \\
 & ADNI3 & 735 \\
 & HeartBrain & 465 \\
\cline{1-3}
\multirow[t]{2}{*}{BOLD} & OASIS3 & 4965 \\
 & rss & 30 \\
\cline{1-3}
\multirow[t]{2}{*}{DTI} & rss & 120 \\
 & ADNI3 & 120 \\
\cline{1-3}
\multirow[t]{2}{*}{DWI} & OASIS3 & 3240 \\
 & egd & 405 \\
\cline{1-3}
FlowSensitve & OASIS3 & 1050 \\
\cline{1-3}
GRE & OASIS3 & 1440 \\
\cline{1-3}
\multirow[t]{2}{*}{Hippo} & ADNI3 & 2955 \\
 & HeartBrain & 225 \\
\cline{1-3}
\multirow[t]{5}{*}{Other} & OASIS3 & 8445 \\
 & ADNI3 & 8370 \\
 & HeartBrain & 1305 \\
 & rss & 90 \\
 & egd & 90 \\
\cline{1-3}
\multirow[t]{2}{*}{PD} & ADNI2 & 420 \\
 & rss & 30 \\
\cline{1-3}
\multirow[t]{3}{*}{PWI} & ADNI3 & 1455 \\
 & egd & 750 \\
 & HeartBrain & 495 \\
\cline{1-3}
\multirow[t]{4}{*}{SWI} & OASIS3 & 1215 \\
 & HeartBrain & 375 \\
 & ADNI3 & 45 \\
 & rss & 30 \\
\cline{1-3}
\multirow[t]{5}{*}{T1